In [ ]:
import math
class Vector3D:
    """
    3D Vector class

    A 3D vector is viewed as a linear combination of
    basis vectors: x*i-hat +  y*j-hat + z*k-hat

    Attributes:
        data: [x,y,z]

    Example:
        >>> v = Vector3D([1,1,2])
    """
    def __init__(self, components):
        if (len(components)) != 3:
            raise ValueError("Must have exactly 3 components")
        self.components = components
        
    def __repr__(self):
        return f"Vector({self.components})"
        
    def __add__(self, other):
        """ Add two vectors component-wise"""
        if len(self.components) != len(other.components):
            raise ValueError("Vectors must be the same dimension")
        result = [a+b for a,b in zip(self.components, other.components)]
        return Vector3D(result)
        
    def __mul__(self, scalar):
        """Multiply vector by a scalar"""
        result = [scalar * component for component in self.components]
        return Vector3D(result)
        
    def magnitude(self):
        """
        Compute the length of the vector.

        The magnitude of the vector determines the
        distance from the  origin.

        Returns:
            float: magnitude
        Formula:
            Pythagorean Theorem: square root(a**2 + b**2 + c**2)
        Example:
            >>> v = Vector3D([3, 4, 0])
            >>> v.magnitude()
            5.0
        """
        # use pythagorean theorem
        sum_of_squares = sum(c**2 for c in self.components)
        return sum_of_squares ** .5
        
    def dot(self, other):
        """Compute dot product with another vector"""
        if len(self.components) != len(other.components):
            raise ValueError("Vectors must be the same dimension")  
        return sum(a*b for a,b in zip(self.components, other.components))
        
    def normalize(self):
        """Return a unit vector (magnitude = 1) in the same direction"""
        return Vector3D([component / self.magnitude() for component in self.components])
        